In [1]:
from Bio import SeqIO
from Bio.SeqFeature import FeatureLocation
from Bio import AlignIO
import re
from glob import glob

In [ ]:
reference = '/Users/Sidney/nextstrain/augur/dengue/metadata/dengue_denv4_outgroup.gb'
want_segments = [('pr', 'M', 'E'), ('NS1', 'NS2A', 'NS2B', 'NS3', 'NS4A', '2K', 'NS4B', 'NS5')]

alignmentfile = '../data/subsampled_alignments/dengue_denv4_aln.fasta'

In [21]:
def load_reference(reference_file):
    reference_seq = SeqIO.read(reference_file, 'genbank')
    gene_locs = {}
    
    genome_annotation = reference_seq.features
    for f in reference_seq.features:
        if 'gene' in f.qualifiers:
            gene_locs[f.qualifiers['gene'][0]] = (int(f.location.start), int(f.location.end))#FeatureLocation(start=f.location.start, end=f.location.end, strand=1)            
    return gene_locs, reference_seq

def convert_coordinate(refseq, compareseq, coordinate):
    '''
    In: ungapped reference sequence, gapped (aligned) compare sequence, position to convert
    Out: coordinate of corresponding condition in the aligned sequence
    '''
    coordinate = coordinate - 1 # Adjust for python coordinates
    #check to make sure we have at least 100bp of downstream sequence, no more than 40bp of which are gaps, to match on
    assert len(refseq[coordinate:]) >= 100 and refseq[coordinate:coordinate+100].count('-')<40, 'ERROR: Not enough downstream context available for '+str(coordinate+1)
    #check to make sure the given coordinate doesn't correspond to a gap in the reference sequence
    assert refseq[coordinate] != '-', 'ERROR! Coordinate '+str(coordinate+1)+' is a gap in the reference sequence.'

    reference = refseq[coordinate:coordinate+100].replace('-', '')
    refpattern = '-*'.join(list(reference)) # Match in new sequence while ignoring gaps
    matchlist = re.findall(refpattern, compareseq, flags=re.IGNORECASE) # Check to make sure we get one and only one match
    assert len(matchlist) == 1, 'ERROR: found %d matches for coordinate %d'%(len(matchlist), coordinate+1)
    return re.search(refpattern, compareseq, flags=re.IGNORECASE).start()+1 #return the converted coordinate (adjusted back to genomic coordinates)

def split_alignment(alignmentobject, segment_locs):
    '''
    in: path to fasta-format alignment file, [(gene, (start,end)), ...]
    out: separate .fasta alignment files for each gene[segment], based on provided coordinates
    NB: removes any sequences without > 70% non-gap sites from each segment alignment
    '''
    ofile_stem = alignmentfile.split('/')[-1].split('.')[0] # name output like alignmentfilename_protein.phyx
    for segment, (start, end) in segment_locs:
        ofile_name = ofile_stem+'_%s.fasta'%(segment)
        
        start, end = start - 1, end - 1 # adjust for pythonic coordinates

        align_segment = alignmentobject[:, start:end+1] #[allrows, startcolumn:endcolumn] endcolumn += 1 for inclusive slicing
        filtered_align_segment = AlignIO.MultipleSeqAlignment([])
        for seq in align_segment:
            seq.seq.data = str(seq.seq).replace('n', '-').replace('N', '-')
            if float(str(seq.seq).count('-')) / float(len(str(seq.seq))) <= 0.30: # Require at least 70% of sites are not gaps to include in segment alignment
                filtered_align_segment.append(seq)

                
                AlignIO.write(filtered_align_segment, ofile_name, 'fasta')

In [24]:
alignmentobject = AlignIO.read(open(alignmentfile, 'r'), 'fasta')

proteins, reference_seq = load_reference(reference) #{'protein': (start, end), ... }, SeqIO.SequenceObject
reference_acc, reference_seq = reference_seq.id.split('.')[0], str(reference_seq.seq) # NB: Expects reference_seq.id like accession.whateverversionignored

if hasattr(want_segments, '__iter__'): # if we want segments composed of multiple adjacent genes....
    tmp = []
    for segment in want_segments:
        name = '_'.join(segment)
        start = min([ proteins[gene][0] for gene in segment ])
        end = max([ proteins[gene][1] for gene in segment])
        tmp.append((name, (start, end)))
    want_segments = tmp

else:
    want_segments = [ (segment, proteins[segment]) for segment in want_segments ]

print want_segments

compare_seq = None
for i in SeqIO.parse(alignmentfile, 'fasta'): # Find the reference sequence in the alignment by matching accession numbers.
    try:
        if i.description.split('|')[1].split('.')[0] == reference_acc:
            compare_seq = str(i.seq)
    except:
        print i.description
assert compare_seq != None

converted_coordinates = [] # Convert coordinates from reference to aligned
for name, loc in want_segments: #[ (protein1, (start, end)), (protein2, (start, end)), ...]
    start = convert_coordinate(reference_seq, compare_seq, loc[0])
    end = convert_coordinate(reference_seq, compare_seq, loc[1])
    converted_coordinates.append((name, (start, end)))

print converted_coordinates

split_alignment(alignmentobject, converted_coordinates)

[('pr_M_E', (440, 2423)), ('NS1_NS2A_NS2B_NS3_NS4A_2K_NS4B_NS5', (2423, 10262))]
[('pr_M_E', (498, 2481)), ('NS1_NS2A_NS2B_NS3_NS4A_2K_NS4B_NS5', (2481, 10320))]


In [27]:
ls

augur_utils/
dengue_denv4_aln_NS1_NS2A_NS2B_NS3_NS4A_2K_NS4B_NS5.fasta
dengue_denv4_aln_pr_M_E.fasta
extract_gene.ipynb
format_titers.py
gard_format.py
generate_tanglegrams.py
geo_colors.ipynb
match_fuzzy_strains.py
mcdonald-kreitman.ipynb
plot_geo_counts.ipynb
remove_undated_sequences.py
untitled
